# Movie Recommendation System / Evaluation and Prediction

This notebook demonstrates the functionality of a movie recommendation system using different recommendation strategies. The system utilizes a combination of collaborative filtering, content-based filtering, and new movie recommendations to provide a comprehensive list of suggestions for users.

## Overview

1. **Loading Data**: The notebook starts by loading cleaned movie and rating data. The data is used to train and evaluate recommendation models.

2. **Loading the Best Model**: An SVD (Singular Value Decomposition) model is loaded from a file. This model has been trained on the data and will be used to generate collaborative filtering recommendations.

3. **Recommendation Functions**:
   - **Collaborative Filtering Recommendations**: Generates movie recommendations based on user ratings and predictions made by the SVD model.
   - **Content-Based Recommendations**: Suggests movies similar to those the user has liked in the past, based on movie genres.
   - **New Movie Recommendations**: Provides a list of newly released or unrated movies that the user might be interested in.
   - **Hybrid Recommendations**: Combines collaborative filtering, content-based filtering, and new movie recommendations to provide a diverse set of suggestions.

## How It Works

1. **Collaborative Filtering**: The SVD model predicts ratings for movies the user has not yet watched. Movies with the highest predicted ratings are recommended.
   
2. **Content-Based Filtering**: Analyzes the genres of movies the user has previously rated and suggests new movies with similar genres that the user hasn't watched.

3. **New Movie Recommendations**: Suggests movies that have not been rated yet, allowing users to explore new content.

4. **Hybrid Recommendations**: Combines the above three strategies to provide a well-rounded list of recommendations.

## Example

In this notebook, we demonstrate how to generate recommendations for a user with `user_id = 1`. The results include:
- Collaborative filtering recommendations
- Content-based recommendations
- New movie recommendations
- A hybrid list combining all the above strategies

Feel free to modify the `user_id` and the number of recommendations to see different results.



In [1]:
# Import necessary libraries
import pandas as pd
import pickle
import numpy as np
from surprise import Dataset, Reader, SVD, accuracy

# Load cleaned data
movies_data = pd.read_csv('data/movies.csv', encoding='latin-1')  # Movie data
ratings_data = pd.read_csv('data/ratings.csv', encoding='latin-1')  # Ratings data

# Load the best model
with open('SVD_model.pkl', 'rb') as file:
    best_model = pickle.load(file)

# Define recommendation functions

def collaborative_filtering_recommendations(user_id, num_recommendations=10):
    """
    Generate movie recommendations based on collaborative filtering.
    
    Collaborative filtering recommendations are made by predicting ratings 
    for all movies that the user has not yet watched and recommending the ones 
    with the highest predicted ratings.
    
    Parameters:
    user_id (int): The ID of the user for whom recommendations are being made.
    num_recommendations (int): The number of top recommendations to return. Default is 10.
    
    Returns:
    list: A list of tuples where each tuple contains a movie ID and the predicted rating.
    """
    all_movie_ids = movies_data['movieId'].unique()
    watched_movies = ratings_data[ratings_data['userId'] == user_id]['movieId']
    recommendations = []
    
    for movie_id in all_movie_ids:
        if movie_id not in watched_movies.values:
            pred = best_model.predict(user_id, movie_id)
            recommendations.append((movie_id, pred.est))
    
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:num_recommendations]

def content_based_recommendations(user_id, num_recommendations=10):
    """
    Generate movie recommendations based on content-based filtering.
    
    Content-based recommendations are made by analyzing the genres of movies 
    that the user has previously rated and suggesting movies with similar genres 
    that the user has not yet watched.
    
    Parameters:
    user_id (int): The ID of the user for whom recommendations are being made.
    num_recommendations (int): The number of top recommendations to return. Default is 10.
    
    Returns:
    list: A list of tuples where each tuple contains a movie ID and the movie title.
    """
    user_ratings = ratings_data[ratings_data['userId'] == user_id]
    user_favorite_genres = movies_data[movies_data['movieId'].isin(user_ratings['movieId'])]['genres'].str.get_dummies(sep='|').sum().sort_values(ascending=False)
    user_favorite_genres = user_favorite_genres[user_favorite_genres > 0].index.tolist()

    recommendations = []
    for _, row in movies_data.iterrows():
        if any(genre in row['genres'] for genre in user_favorite_genres):
            if row['movieId'] not in user_ratings['movieId'].values:
                recommendations.append((row['movieId'], row['title']))
    
    return recommendations[:num_recommendations]

def new_movie_recommendations(num_recommendations=5):
    """
    Generate recommendations for movies that have not been rated yet.
    
    This function returns a list of newly released or unrated movies for 
    users who may be interested in exploring new content.
    
    Parameters:
    num_recommendations (int): The number of top recommendations to return. Default is 5.
    
    Returns:
    list: A list of lists where each inner list contains a movie ID and the movie title.
    """
    new_movies = movies_data[~movies_data['movieId'].isin(ratings_data['movieId'])]
    return new_movies[['movieId', 'title']].head(num_recommendations).values.tolist()

def hybrid_recommendations(user_id, num_collaborative=5, num_content=3, num_new=2):
    """
    Generate a hybrid recommendation list combining collaborative filtering, 
    content-based filtering, and new movie recommendations.
    
    Parameters:
    user_id (int): The ID of the user for whom recommendations are being made.
    num_collaborative (int): The number of collaborative filtering recommendations to include. Default is 5.
    num_content (int): The number of content-based recommendations to include. Default is 3.
    num_new (int): The number of new movie recommendations to include. Default is 2.
    
    Returns:
    list: A combined list of recommendations from collaborative filtering, content-based filtering, and new movies.
    """
    collab_recs = collaborative_filtering_recommendations(user_id, num_collaborative)
    content_recs = content_based_recommendations(user_id, num_content)
    new_recs = new_movie_recommendations(num_new)
    recommendations = collab_recs + content_recs + new_recs
    return recommendations

# Generate recommendations
user_id = 1  # Example user ID
print("Collaborative Filtering Recommendations:")
print(collaborative_filtering_recommendations(user_id))

print("\nContent-Based Recommendations:")
print(content_based_recommendations(user_id))

print("\nNew Movie Recommendations:")
print(new_movie_recommendations())

print("\nHybrid Recommendations:")
print(hybrid_recommendations(user_id))


Collaborative Filtering Recommendations:
[(3435, 4.907418202093723), (950, 4.836257001243191), (527, 4.831996057516722), (2905, 4.794858069673878), (3077, 4.754768155530078), (904, 4.7491160808810715), (3679, 4.739693269306485), (2937, 4.7291097975200955), (899, 4.712275894088054), (1212, 4.699696691943146)]

Content-Based Recommendations:
[(2, 'Jumanji (1995)'), (3, 'Grumpier Old Men (1995)'), (4, 'Waiting to Exhale (1995)'), (5, 'Father of the Bride Part II (1995)'), (6, 'Heat (1995)'), (7, 'Sabrina (1995)'), (8, 'Tom and Huck (1995)'), (9, 'Sudden Death (1995)'), (10, 'GoldenEye (1995)'), (11, 'American President, The (1995)')]

New Movie Recommendations:
[[52696, 'Thousand and One Nights, A (1001 Nights) (1945)'], [58209, 'Alex in Wonder (Sex and a Girl) (2001)'], [69864, 'Blue Blood (2006)'], [70312, 'Family Secret, A (Le secret de ma mÃ¨re) (2006)'], [71355, 'February (Khumphaphan) (2003)']]

Hybrid Recommendations:
[(3435, 4.907418202093723), (950, 4.836257001243191), (527, 4.83